# Check Conda Env Name

In [1]:
import sys
sys.executable.split('/')[-3]

# Result Filename

In [2]:
result_filename='alibaba_CelebDPR.csv'

# Imports

In [3]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.face import FaceClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from array import array
import os
import sys
import time
from PIL import Image
import csv
from time import sleep
from glob import glob
from pprint import pprint

In [6]:
import sys
print (sys.path)
print(os. getcwd())

In [8]:
#coding=utf-8
# The ImageSyncScanRequest operation returns image moderation results in real time.

from aliyunsdkcore import client
from aliyunsdkgreen.request.v20180509 import ImageSyncScanRequest
from aliyunsdkgreen.request.extension import ClientUploader
from aliyunsdkgreenextension.request.extension import HttpContentHelper
import json
import uuid
import sys

# Set the default encoding to UTF-8 to support a local path containing Chinese characters.
# Add the following code snippet if you use Python 2. If you use Python 3, remove it.
if sys.version_info[0] == 2:
    reload(sys)
    sys.setdefaultencoding('utf-8')

# Use the AccessKey ID and AccessKey secret of your Alibaba Cloud account.
clt = client.AcsClient("", "", "")
request = ImageSyncScanRequest.ImageSyncScanRequest()
request.set_accept_format('JSON')

# Use the path of your local image.
# Upload the local image to the server.
uploader = ClientUploader.getImageClientUploader(clt)
url = uploader.uploadFile('Similarity/CelebDPR/real/Scarlett Johansson.jpg')

# Set the url parameter to the image URL submitted to the server.
task = {"dataId": str(uuid.uuid1()),
         "url":url
        }

# Create one task for each image to be moderated.
# If you moderate multiple images in a request, the total response time that the server spends processing the request starts from when the request is initiated to when the last image is moderated.
# Generally, the average response time of moderating multiple images in a request is longer than that of moderating a single image. The more images you submit at a time, the higher the probability that the average response time will be extended.
# The sample code uses a single image as an example. If you want to moderate multiple images at a time, create one task for each image to be moderated.
# The system charges you based on the moderation scenario that you specify.
# You can send a request to moderate multiple images at a time and specify multiple moderation scenarios for each image. The expenses of all scenarios are calculated separately and summed up.
# For example, if you moderate two images for both pornography and terrorist content, you are charged for moderating two images for pornography and two images for terrorist content.
request.set_content(HttpContentHelper.toValue({"tasks": [task], "scenes": ["porn"]}))
response = clt.do_action_with_exception(request)
print(response)
result = json.loads(response)
if 200 == result["code"]:
    taskResults = result["data"]
    for taskResult in taskResults:
        if (200 == taskResult["code"]):
            sceneResults = taskResult["results"]
            for sceneResult in sceneResults:
                # Take a further action on the image based on the values of the scene and suggestion parameters.
                # do something
                scene = sceneResult["scene"]
                suggestion = sceneResult["suggestion"]

## API Setting 

In [4]:
subscription_key = "your key"
endpoint="your endpoint"
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
subscription_key = "your key"
endpoint="your endpoint"
face_client=FaceClient(endpoint, CognitiveServicesCredentials(subscription_key))

## CelebDPR

### Describe an Image

In [5]:
'''
Describe an Image - local
This example describes the contents of an image with the confidence score.'Scarlett Johansson_fake.png'
'''
def describe_image(filename):
#     print("===== Describe an image - local =====")
    # Call API
    image= open(filename,"rb")
    description_results = computervision_client.describe_image_in_stream(image)

    # Get the captions (descriptions) from the response, with confidence level
#     print("Description of local image: ")
    if (len(description_results.captions) == 0):
#         print("No description detected.")
        return "No description detected."
    else:
        for caption in description_results.captions:
#             print("'{}' with confidence {:.2f}%".format(caption.text, caption.confidence * 100))
            return "'{}' with confidence {:.2f}%".format(caption.text, caption.confidence * 100)

### Detect Domain-specific Content

In [6]:
'''
Detect Domain-specific Content - local
This example detects celebrites and landmarks in local images.
'''
def domain_specific_content(filename):
#     print("===== Detect Domain-specific Content - local =====")
    # Image of one or more celebrities
    image= open(filename,"rb")
    # image = image.read('Scarlett.png')

    # Call API with content type (celebrities) and URL
    detect_domain_results_celebs_remote = computervision_client.analyze_image_by_domain_in_stream("celebrities", image)
    # Print detection results with name
#     print("Celebrities in the local image:")
    if len(detect_domain_results_celebs_remote.result["celebrities"]) == 0:
        print("No celebrities detected.")
        return "",""
    else:
        for celeb in detect_domain_results_celebs_remote.result["celebrities"]:
#             print(celeb["name"])
#             print(celeb["confidence"])
            return celeb["name"],celeb["confidence"]

In [7]:
def get_name(filename):
    return os.path.splitext(os.path.basename(filename))[0].split("_")[0]

In [8]:
def check_similarity(src_image, tar_image):
    # Detect a face in an image that contains a single face
    src_image_name = os.path.basename(src_image)
    detected_faces = face_client.face.detect_with_stream(open(src_image,"rb"))
    if not detected_faces:
        print('No face detected from image {}'.format(single_image_name))
        return 0
#         raise Exception('No face detected from image {}'.format(single_image_name))

    # Display the detected face ID in the first single-face image.
    # Face IDs are used for comparison to faces (their IDs) detected in other images.
#     print('Detected face ID from', single_image_name, ':')
#     for face in detected_faces: 
#         print (face.face_id)
#     print()

    # Save this ID for use in Find Similar
    first_image_face_ID = detected_faces[0].face_id
    
    multi_image_name = os.path.basename(tar_image)
    detected_faces2 = face_client.face.detect_with_stream(open(tar_image,"rb"))
    if not detected_faces2:
        print('No face detected from image {}'.format(multi_image_name))
        return 0
#         raise Exception('No face detected from image {}'.format(multi_image_name))
    
    # Search through faces detected in group image for the single face from first image.
    # First, create a list of the face IDs found in the second image.
    second_image_face_IDs = list(map(lambda x: x.face_id, detected_faces2))
    # Next, find similar face IDs like the one detected in the first image.
    similar_faces = face_client.face.find_similar(face_id=first_image_face_ID, face_ids=second_image_face_IDs)
    
    if not similar_faces:
        print('No similar faces found in', multi_image_name, '.')
        return 0
    else:
        # Print the details of the similar faces detected
#         print('Similar faces found in', multi_image_name + ':')
        for face in similar_faces:
        #     print(face)
            return face.confidence
#             print('  Confidence:', face.confidence)
#             first_image_face_ID = face.face_id
#             # The similar face IDs of the single face image and the group image do not need to match, 
#             # they are only used for identification purposes in each image.
#             # The similar faces are matched using the Cognitive Services algorithm in find_similar().
#             face_info = next(x for x in detected_faces2 if x.face_id == first_image_face_ID)
#             if face_info:
#                 print('  Face ID: ', first_image_face_ID)
#                 print('  Face rectangle:')
#                 print('    Left: ', str(face_info.face_rectangle.left))
#                 print('    Top: ', str(face_info.face_rectangle.top))
#                 print('    Width: ', str(face_info.face_rectangle.width))
#                 print('    Height: ', str(face_info.face_rectangle.height))

In [12]:
file=open(result_filename, 'w', newline='')
writer = csv.writer(file)
writer.writerow(["Filename", "Src","Tar", "Src Pred.", "Src Conf.", "Tar Pred.","Tar Conf.", "Similarity","Src Des", "Tar Des"])
file.close()
actors_list=glob("/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/*")
count=0
skip=0
for actor in actors_list:
    count+=1
#     if skip ==0:
#         if os.path.basename(actor) =="June Lapine Aka Shoe0nHead":
#             skip=1
#         else:
#             continue
#     print(os.path.basename(actor))
#     break
    
    real_image=os.path.join("/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/real",
                                os.path.basename(actor)+".jpg")
    fake_images_list=glob(os.path.join(actor,"*"))
    print(real_image)
#     pprint(fake_images_list)
#     continue
    result_real_pred,result_real_conf=domain_specific_content(real_image)
    real_des=describe_image(real_image)
    src=get_name(real_image)
    for fake_image in fake_images_list:
        print(fake_image)
#         continue
        result_fake_pred,result_fake_conf=domain_specific_content(fake_image)
        fake_des=describe_image(fake_image)
        tar=get_name(fake_image)
        similarity=check_similarity(real_image,fake_image)
#             print(fake_image,src,tar,
#                   result_real_pred,result_real_conf,
#                   result_fake_pred,result_fake_conf,
#                   similarity,real_des,fake_des)
        file=open(result_filename, 'a', newline='')
        writer = csv.writer(file)
        writer.writerow([fake_image,src,tar,
              result_real_pred,result_real_conf,
              result_fake_pred,result_fake_conf,
              similarity,real_des,fake_des])
        file.close()
#             sleep(10)
#             break
#         break
#     break
# file.close()
print("Total:",count)